In [38]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import os
import json


In [3]:
path_unq_bks = "/home/aca/Documents/S6_Lab/Projects/libraryDataAnalysis/venv/recomm/rss/unq_bks2.csv"


In [42]:
def combine_features(data=None, valid_args=None):

    if (data is None or valid_args is None):
        print("No dataframe selected/No arguments passed to combine")
        return

    _keys = list(valid_args.keys())

    # If you're passing scalar values, you have to pass an index
    # https://stackoverflow.com/a/17840195/12616968
    _row = pd.DataFrame(valid_args, index=[0])

    data = pd.concat([data, _row], axis=0, ignore_index=True)

    features = []

    sz = data.shape[0]

    try:
        for i in range(sz):
            fea = ""
            for j in _keys:
                fea += data[j][i] + " "
            features.append(fea)
    except:
        print(i)
    finally:
        data["combined"] = features
        return data


In [49]:
def search_similar(auth="", pub="", title="", course_code = ""):

    lst = []

    valid_args = {"author": "","title": "", "publisher": ""}

    if (title != ""):
        valid_args["title"] = title
    if (pub != ""):
        valid_args["publisher"] = pub
    if (auth != ""):
        valid_args["author"] = auth

    if (not valid_args):
        return ""

    data = pd.read_csv(path_unq_bks, index_col=0)

    if (data is None):
        return "<div class='alert alert-danger'>No data frame selected</div>"

    _temp = combine_features(
        data=data[list(valid_args.keys())], valid_args=valid_args)

    cm = CountVectorizer().fit_transform(_temp["combined"])

    # get cosine similarity mtx
    cs = cosine_similarity(cm)

    index = cs.shape[0]-1  # index of the added row

    a = list(enumerate((cs[index])))

    # Sort scores in descending order. More score means higher similarity
    sorted_scores = sorted(a, key=lambda x: x[1], reverse=True)

    count = 0
    for i in sorted_scores[1:]:
        if (i[1] <= 0.499):
            continue
        try:
            _temp = {}

            _temp["book_id"] = int(i[0])
            _temp["title"] = data[data.index == i[0]]['title'].values[0]
            _temp["author"] = data[data.index == i[0]]['author'].values[0]
            _temp["publisher"] = data[data.index ==
           💡                           i[0]]['publisher'].values[0]
            _temp["number_of_books"] = int(data[data.index ==
                                            i[0]]['number_of_books'].values[0])
            _temp["score"] = float(i[1])
            _temp["course_code"] = course_code

            lst.append(_temp)


            count += 1

            if count >= 3: # Max recommendation per book title
                break

        except IndexError:
            continue

    return lst


In [48]:
input_path = "/home/aca/Documents/S6_Lab/Projects/libraryDataAnalysis/model/feeling_lucky/textbooks"
output_path = "/home/aca/Documents/S6_Lab/Projects/libraryDataAnalysis/model/feeling_lucky/recommended_bks/"
files = os.listdir(input_path)


In [52]:
for file in files:
    _t = {}
    with open(os.path.join(input_path, file)) as f:
        _ = f.read().strip().split("\n")
        for j in _:
            j = j.upper()
            _t[j] = search_similar(title = j, course_code = file[:-4])
    
    with open(os.path.join(output_path, file[:-4] + ".json"), 'w') as _f:
        json.dump(_t, _f)
    

